In [1]:
import gensim
import json
import re
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim 

morph = MorphAnalyzer()

C:\Users\Eolain\Anaconda3\envs\mllecture\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Домашнее задание

Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками**.

1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); 

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases); 

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); 

4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

6) между словарем и обучением модели добавьте tfidf (`gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]`);

7) заново обучите LDA c теми же параметрами (параметрами самой лучшей модели, заново перебирать не нужно);

8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

10) в отдельной ячейки напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA.

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 5, 8 и 10. (2, 3, 2 баллов соответственно). Чтобы заработать остальные 3 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **

## Данные

In [2]:
def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def clean(words):
    clean = [morph.parse(word)[0].normal_form for word in words if len(set(word)) > 1]
    return clean

Возьмем 4 тыс статьи с Хабра. Это мало для хорошей тематической модели, но иначе у нас просто ничего не обучится за семинар.

In [4]:
texts = open('habr_texts.txt', encoding='utf-8').read().splitlines()

В текстах есть тэги. Потрем их. Ещё токенизируем самым простым способом и нормализуем Pymorphy.

In [5]:
texts = [clean(word_tokenize(text.lower())) for text in texts]

In [11]:
# для нграммов
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на', 'и', 'с'])
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.44, common_terms=set(stop_words)) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

C:\Users\Eolain\Anaconda3\envs\mllecture\lib\site-packages\gensim\models\phrases.py:317: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if all(parameter in getargspec(scoring)[0] for parameter in scoring_parameters):
C:\Users\Eolain\Anaconda3\envs\mllecture\lib\site-packages\gensim\models\phrases.py:317: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if all(parameter in getargspec(scoring)[0] for parameter in scoring_parameters):
C:\Users\Eolain\Anaconda3\envs\mllecture\lib\site-packages\gensim\models\phrases.py:317: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if all(parameter in getargspec(scoring)[0] for parameter in scoring_parameters):
C:\Users\Eolain\Anaconda3\envs\mllecture\lib\site-packages\gensim\models\phrases.py:317: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if all(parameter in getargspec(scoring)[0

In [18]:
p[texts[1]]

['введение',
 'выбор',
 'решение',
 'рано_или_поздно',
 'наступать',
 'такой',
 'момент',
 'жизнь',
 'любой',
 'сообщество',
 'форум',
 'когда',
 'для',
 'привлечение_удержание',
 'человек',
 'возникать',
 'острый_необходимость',
 'использование',
 'новое',
 'инструментарий',
 'такой',
 'весьма',
 'эффективный',
 'инструмент',
 'являться',
 'то',
 'что',
 'крыться',
 'за',
 'модный_ныне',
 'слово',
 'геймификация',
 'то',
 'есть',
 'использование',
 'характерный',
 'для',
 'игра',
 'приём',
 'подход',
 'неигровой',
 'процесс',
 'по',
 'привлечение',
 'вовлечение',
 'участник',
 'сообщество',
 'на',
 'форум',
 'создание',
 'активный',
 'мощный',
 'информационный',
 'поль',
 'вокруг',
 'наш',
 'продукт',
 'использовать',
 'наш',
 'компания',
 'форумный_движок',
 'xenforo_настоящее',
 'время',
 'являться',
 'наиболее_популярный',
 'быстро',
 'развивающийся',
 'этот',
 'движок',
 'по-умолчание',
 'иметь',
 'встроить',
 'система',
 'трофей_основать',
 'на',
 'собственный',
 'весьма',
 'огра

### Тематическое моделирование в gensim

Для моделей нужно сделать словарь.

In [20]:
dictinary = gensim.corpora.Dictionary(ngrammed_texts)

In [25]:
dictinary.filter_extremes(no_above=0.25, no_below=25)
dictinary.compactify()

In [26]:
print(dictinary)

Dictionary(5737 unique tokens: ['примечание', 'ежедневно', 'семь', 'фокус', 'подразделение']...)


Преобразуем наши тексты в мешки слов. 

In [27]:
corpus = [dictinary.doc2bow(text) for text in texts]
# если текстов много, то тут может быть генератор

In [50]:
lda = gensim.models.LdaMulticore(corpus, 250, id2word=dictinary, passes=5) # если поддерживается многопоточность
# lsi = gensim.models.LdaModel(200, id2word=dictinary, passes=5)

C:\Users\Eolain\Anaconda3\envs\mllecture\lib\site-packages\gensim\models\ldamodel.py:802: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


Посмотрим на топики.

In [97]:
lda.print_topics(num_topics=-1)[122]

(122,
 '0.017*"двигатель" + 0.014*"ракета" + 0.013*"аппарат" + 0.013*"спутник" + 0.010*"запуск" + 0.010*"космический" + 0.010*"ступень" + 0.010*"орбита" + 0.010*"полёт" + 0.009*"корабль"')

In [99]:
lda.print_topics(num_topics=-1)[0]

(0,
 '0.017*"безопасность" + 0.016*"защита" + 0.012*"пароль" + 0.012*"атака" + 0.007*"security" + 0.007*"google" + 0.007*"уязвимость" + 0.005*"эксплуатация" + 0.005*"скрипт" + 0.005*"доступ"')

In [102]:
lda.print_topics(num_topics=-1)[242]

(242,
 '0.031*"процессор" + 0.019*"видеокарта" + 0.014*"память" + 0.012*"производительность" + 0.010*"игра" + 0.009*"nvidia" + 0.009*"fps" + 0.009*"железо" + 0.008*"архитектура" + 0.008*"intel"')

Ещё есть штука для визуализации.

In [63]:
pyLDAvis.enable_notebook()

In [64]:
pyLDAvis.gensim.prepare(lda, corpus, dictinary)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


Можно посмотреть метрики.

In [67]:
import numpy as np

In [68]:
lda.log_perplexity(corpus[:2000], total_docs=100)

-74.77953844734222

In [70]:
coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [71]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

In [72]:
coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [73]:
coherence_model_lda.get_coherence()

0.4472107961290014

In [104]:
# модель с tfidf
tfidf = gensim.models.TfidfModel(corpus, id2word=dictinary)
corpus = tfidf[corpus]

In [105]:
lda = gensim.models.LdaMulticore(corpus, 250, id2word=dictinary, passes=5)

C:\Users\Eolain\Anaconda3\envs\mllecture\lib\site-packages\gensim\models\ldamodel.py:802: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [107]:
lda.print_topics(num_topics=-1)[122]

(122,
 '0.048*"смартфон" + 0.034*"аккумулятор" + 0.023*"дисплей" + 0.020*"телефон" + 0.016*"розетка" + 0.015*"крышка" + 0.012*"батарея" + 0.012*"оснастить" + 0.011*"батарейка" + 0.011*"гаджет"')

In [108]:
lda.print_topics(num_topics=-1)[0]

(0,
 '0.006*"flash" + 0.005*"герой" + 0.005*"днк" + 0.005*"пыль" + 0.004*"виджет" + 0.004*"бэкап" + 0.003*"in" + 0.003*"вселенная" + 0.003*"автомобильный" + 0.002*"память"')

In [109]:
lda.print_topics(num_topics=-1)[242]

(242,
 '0.022*"мастер" + 0.015*"след" + 0.005*"сенсорный" + 0.005*"length" + 0.004*"лазер" + 0.004*"луна" + 0.004*"local" + 0.004*"поверхность" + 0.003*"skype" + 0.003*"машина"')

In [110]:
lda.log_perplexity(corpus[:2000], total_docs=100)

-346.4470521550176

In [111]:
coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [112]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

In [113]:
coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [114]:
coherence_model_lda.get_coherence()

0.34835088966629985

In [ ]:
### После добавления модели tfidf упали обе метрики perplexity и coherence.

### Разложение матриц в sklearn

In [39]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

Sklearn принимает на вход строки, поэтому склеим наши списки.

In [40]:
stexts = [' '.join(text) for text in texts]

Сделаем матрицу слова-документы с помощью TfidfVectorizer

In [41]:
vectorizer = TfidfVectorizer(max_features=25000, min_df=5, max_df=0.3, lowercase=False)
X = vectorizer.fit_transform(stexts)

C:\Users\Eolain\Anaconda3\envs\mllecture\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Разложим её.

In [42]:
model = NMF(n_components=30)

In [43]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=30, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [46]:
def get_nmf_topics(model, n_top_words):
    
    #id слов.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(30):
        
        #топ n слов для темы.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic #' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [47]:
get_nmf_topics(model, 10)

,Topic #01,Topic #02,Topic #03,Topic #04,Topic #05,Topic #06,Topic #07,Topic #08,Topic #09,Topic #10,...,Topic #21,Topic #22,Topic #23,Topic #24,Topic #25,Topic #26,Topic #27,Topic #28,Topic #29,Topic #30
0,ты,gt,игра,космический,объект,сеть,камера,js,сайт,файл,...,язык,книга,public,центр,бот,робот,товар,windows,звук,if
1,продукт,lt,игрок,спутник,значение,трафик,устройство,react,страница,docker,...,программирование,часы,void,дата,telegram,ребёнок,рынок,microsoft,сигнал,return
2,сотрудник,div,игровой,орбита,элемент,связь,смартфон,css,реклама,php,...,php,каковать,string,облачный,сообщение,автомобиль,рубль,linux,наушник,function
3,бизнес,class,играть,ракета,блок,ip,аккумулятор,javascript,клиент,sudo,...,программист,порекомендовать,new,инфраструктура,телеграм,lego,российский,studio,усилитель,amp
4,клиент,this,vr,марс,метод,оператор,телефон,angular,контент,nginx,...,лекция,профессиональный,private,услуга,чат,датчик,россия,visual,частота,null
5,программист,name,персонаж,аппарат,класс,интернет,дисплей,веб,браузер,http,...,java,каким,класс,облако,bot,робототехника,скидка,net,звуковой,end
6,менеджер,props,steam,земля,строка,устройство,гб,vue,письмо,сервер,...,перевод,предпочитать,class,оборудование,api,машина,магазин,azure,музыка,self
7,опыт,return,unity,луна,int,dpi,экран,браузер,домен,etc,...,программа,бумажный,int,цод,мессенджер,движение,страна,браузер,искажение,else
8,да,std,геймплей,наса,файл,канал,xiaomi,компонент,рекламный,скрипт,...,python,слушать,static,сервис,канал,беспилотный,цена,обновление,акустический,var
9,деньга,html,движок,полёт,алгоритм,адрес,видео,dom,google,usr,...,курс,путь,return,ит,message,дрон,налог,server,напряжение,this
